In [2]:
#!/usr/bin/evn python

"""
CMSC733 Spring 2019: Classical and Deep Learning Approaches for
Geometric Computer Vision
Project1: MyAutoPano: Phase 1 Starter Code

Author(s): 
Chahat Deep Singh (chahat@terpmail.umd.edu) 
PhD Student in Computer Science,
University of Maryland, College Park

Nitin J. Sanket (nitinsan@terpmail.umd.edu)
PhD Candidate in Computer Science,
University of Maryland, College Park
"""

#Code starts here:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import peak_local_max
import argparse

In [3]:
def readImageSet(folder_name):
    print("Reading images from ", folder_name)
    images = []
    files = os.listdir(folder_name)	
    files = sorted(files)
    print("Found ", files)
    for file in files:
        image_path = folder_name + "/" + file
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)			
        else:
            print("Error in loading image ", image)

    return images


In [4]:
def displayImages(img_array, file_name):
    
    image_array = img_array.copy()
    image_array = makeImageSizeSame(image_array)
    concat = image_array[0].copy()

    for l in range(1,len(image_array)):
        image = image_array[l]
        concat = np.concatenate((concat,image), axis = 1)
        
    cv2.imshow(file_name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()
    #cv2.imwrite(file_name, concat)


In [5]:
def detectCorners(imgs, choice):
    images = imgs.copy()
    print("detecting corners ...")
    detected_corners = []
    cmaps = []
    corner_images = []
    for i in images:
        image = i.copy()
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray_image)


        if(choice == 1):
            print("using Harris corner detection method.")
            corner_strength = cv2.cornerHarris(gray_image,2,3,0.04)
            corner_strength[corner_strength<0.01*corner_strength.max()] = 0
            detected_corner = np.where(corner_strength>0.001*corner_strength.max())
            detected_corners.append(detected_corner)
            cmaps.append(corner_strength)
            image[corner_strength > 0.001*corner_strength.max()]=[0,0,255]
            corner_images.append(image)
        else:
            print("using Shi-Tomashi corner detection method.")
            dst = cv2.goodFeaturesToTrack(gray_image, 1000 ,0.01, 10)
            dst = np.int0(dst)
            detected_corners.append(dst)
            for c in dst:
                x,y = c.ravel()
                cv2.circle(image,(x,y),3,(0, 0, 255),-1) 
                          
            corner_images.append(image)
            cmap = np.zeros(gray_image.shape) #not sure what to do
            cmaps.append(cmap)
    #filter detected corners
    #remove the corner one
    return detected_corners, cmaps, corner_images

In [6]:
def getFeatureDescriptor(gray_img,x,y, patch_size=40):
    gray_image = gray_img
    patch = gray_image[x-patch_size//2:x+patch_size//2, y-patch_size//2:y+patch_size//2] 
    # gaussian blur
    patch = cv2.GaussianBlur(patch,(3,3),0)
    # subsample to 20% size or 1/5th
    patch = cv2.resize(patch, None, fx=0.2, fy=0.2, interpolation = cv2.INTER_CUBIC)
    feature = patch.reshape(-1)
    feature = (feature-feature.mean())/ np.std(feature)
    return feature

In [7]:
def getPairs(img_1, img_2, all_corners_1, all_corners_2, patch_size = 40, alpha = 0.8):

    image_1 = img_1.copy()
    image_2 = img_2.copy()

    gray_image1 = cv2.cvtColor(image_1,cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image_2,cv2.COLOR_BGR2GRAY)

    width1, height1 = image_1.shape[:2]
    width2, height2 = image_2.shape[:2]

    print("width = ", width1, ", height = ", height1)
    print("width = ", width2, ", height = ", height2)

    features_1,features_2 = [], []
    corners_1, corners_2 = [],[]

    print("all corner 1 len = ", len(all_corners_1))
    print("all corner 2 len = ", len(all_corners_2))
    
    for corner in all_corners_1:
        x,y = corner.ravel()
        
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height1) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width1):
            features_1.append(getFeatureDescriptor(gray_image1, y,x)) 
            corners_1.append([x,y])
        else:
            #print("ignored x, y", x, y)
            pass

    for corner in all_corners_2:
        x,y = corner.ravel()
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height2) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width2):
            features_2.append(getFeatureDescriptor(gray_image2, y,x)) 
            corners_2.append([x,y]) 

    matched_pairs, match_level = [], []
    for i, feat_1 in enumerate(features_1):
        ssd = []  
        for j, feat_2 in enumerate(features_2):
            ssd.append(np.sum((feat_1 - feat_2)**2))
        top_matche = np.argmin(ssd)
        if ssd[top_matches[0]] / ssd[top_matches[1]] < alpha:   
            matched_pairs.append([corners_1[i] , corners_2[top_matches[0]]])
        matched_pairs.append([corners_1[i] , corners_2[top_matche]]) 
    print("matched pairs num = ", len(matched_pairs))
    matched_pairs = np.array(matched_pairs)
    return matched_pairs

In [8]:
def makeImageSizeSame(imgs):
    images = imgs.copy()
    sizes = []
    for image in images:
        x, y, ch = image.shape
        sizes.append([x, y, ch])

    sizes = np.array(sizes)
    x_target, y_target, _ = np.max(sizes, axis = 0)
    
    images_resized = []

    for i, image in enumerate(images):
        image_resized = np.zeros((x_target, y_target, sizes[i, 2]), np.uint8)
        image_resized[0:sizes[i, 0], 0:sizes[i, 1], 0:sizes[i, 2]] = image
        images_resized.append(image_resized)

    return images_resized

In [9]:
def showMatches(img_1, img_2, matched_pairs, file_name):

    image_1 = img_1.copy()
    image_2 = img_2.copy()

    image_1, image_2 = makeImageSizeSame([image_1, image_2])

    concat = np.concatenate((image_1, image_2), axis = 1)
    corners_1 = matched_pairs[:,0].copy()
    corners_2  = matched_pairs[:,1].copy()
    corners_2[:,0] += image_1.shape[1]

    for (x1,y1) , (x2,y2) in zip(corners_1, corners_2):
        cv2.line(concat, (x1,y1), (x2,y2), (0, 0, 255), 1)
    
      
    cv2.imshow(file_name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()
    cv2.imwrite(file_name, concat)


In [10]:
def testShowMatches(image_1, image_2, partition_width = 20):
    matched_pairs= []
    I = np.linspace(10, 100, 10)
    for i in I:
        x1 = i
        y1 = i
        corner1 = np.int0(np.array([x1, y1]))

        x2 = i
        y2 = i
        corner2 = np.int0(np.array([x2, y2]))
        matched_pairs.append([corner1 , corner2])

    matched_pairs = np.array(matched_pairs)
    showMatches(image_1, image_2, matched_pairs, partition_width = 20)


In [11]:
def filterOutliers(matched_pairs, outliers, accuracy, thresh):

    set1 = matched_pairs[:, 0]
    set2 = matched_pairs[:, 1]

    N_best = 0
    H_best = np.zeros([3, 3])
    
    e = outliers / set1.shape[0]
    s = 4
    p = accuracy
    iterations = np.log(1 - p) / np.log(1 - np.power((1 - e), s))
    iterations = np.int(iterations)
    iterations = 4000

    filtered_pair_indices = []

    print("iterations = ", iterations)
    for i in range(iterations):
        #randomly select four points
        n_rows = set1.shape[0]
        random_indices = np.random.choice(n_rows, size=4)

        set1_random = set1[random_indices]
        set2_random = set2[random_indices]
              
        #compute homography
        H = cv2.getPerspectiveTransform(np.float32(set1_random), np.float32(set2_random))

        set1_dash = np.vstack((set1[:,0], set1[:,1], np.ones([1, n_rows])))
        set1_transformed_dash = np.dot(H, set1_dash)
        
        t1 = set1_transformed_dash[0,:]/set1_transformed_dash[2,:]
        t2 = set1_transformed_dash[1,:]/set1_transformed_dash[2,:]

        set1_transformed = np.array([t1, t2]).T
        #print(set1_transformed.shape)

        E = calculateError(set2, set1_transformed)
        
     
        E[E <= thresh] = 1
        E[E > thresh] = 0
    
    
        N = np.sum(E)


        if N > N_best:
            N_best = N
            H_best = H
            filtered_pair_indices = np.where(E == 1)
    
    filtered_set1 =  set1[filtered_pair_indices]
    filtered_set2 =  set2[filtered_pair_indices]

    print("Number of pairs after filtering = ", filtered_set1.shape[0])

    filter_matched_pairs = np.zeros([filtered_set1.shape[0], filtered_set1.shape[1], 2])

    filter_matched_pairs[:, 0, :] = filtered_set1
    filter_matched_pairs[:, 1, :] = filtered_set2

    filter_matched_pairs = filter_matched_pairs.astype(int)

    H_inbuit, _ = cv2.findHomography(set1, set2)
    print("inbuilt = ", H_inbuit)
    print("Computed = ", H_best)

    return H_best, filter_matched_pairs

    

In [12]:
def calculateError(set1, set2):
   
    E = np.zeros(set1.shape[0])
    tmp = set2 - set1
    num = set1.shape[0]

    for n in range(num):
        E[n] = np.linalg.norm(tmp[n])
    return E


In [13]:
def AdaptiveNonMaximalSuppression(imgs, C_maps, N_best):
    
    images = imgs.copy()
    anms_img = []
    anms_corners = []
    for i,image in enumerate(images):

        cmap = C_maps[i]
        local_maximas = peak_local_max(cmap, min_distance=10)
        n_strong = local_maximas.shape[0]
        
        r = [np.Infinity for i in range(n_strong)]
        x=np.zeros((n_strong,1))
        y=np.zeros((n_strong,1))
        eu_dist = 0

        for i in range(n_strong):
            for j in range(n_strong):
                x_j = local_maximas[j][0]
                y_j = local_maximas[j][1]

                x_i = local_maximas[i][0]
                y_i = local_maximas[i][1]

                if(cmap[x_j, y_j] > cmap[x_i, y_i]):
                    eu_dist = np.square(x_j - x_i) + np.square(y_j - y_i)
                if r[i] > eu_dist:
                    r[i] = eu_dist
                    x[i] = x_j
                    y[i] = y_j

        index = np.argsort(r)
        index = np.flip(index)
        index = index[0:N_best]
        x_best=np.zeros((N_best,1))
        y_best=np.zeros((N_best,1))

        for i in range(N_best):
            x_best[i] = np.int0(x[index[i]])
            y_best[i] = np.int0(y[index[i]]) 
            cv2.circle(image, (y_best[i], x_best[i]), 3, (0, 255, 0), -1)

        anms_corner = np.int0(np.concatenate((x_best, y_best), axis = 1))
        anms_corners.append(anms_corner)
        anms_img.append(image)
    return anms_corners, anms_img

In [14]:
def stitchImagePairs(img0, img1, H):

    image0 = img0.copy()
    image1 = img1.copy()

    #stitch image 0 on image 1
    print("shapes")
    print(image0.shape)
    print(image1.shape)
    

    h0 ,w0 ,_ = image0.shape
    h1 ,w1 ,_ = image1.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)
    print("transformed points = ", points_on_image0_transformed)
    points_on_image1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1,1,2)

    points_on_merged_images = np.concatenate((points_on_image0_transformed, points_on_image1), axis = 0)
    points_on_merged_images_ = []

    for p in range(len(points_on_merged_images)):
        points_on_merged_images_.append(points_on_merged_images[p].ravel())

    points_on_merged_images_ = np.array(points_on_merged_images_)

    x_min, y_min = np.int0(np.min(points_on_merged_images_, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_merged_images_, axis = 0))

    print("min, max")
    print(x_min, y_min)
    print(x_max, y_max)

    # overlap_area = cv2.polylines(image1,[np.int32(points_on_image0_transformed)],True,255,3, cv2.LINE_AA) 
    # cv2.imshow("original_image_overlapping.jpg", overlap_area)
    # cv2.waitKey() 
    # cv2.destroyAllWindows()
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed_and_stitched = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    image0_transformed_and_stitched[-y_min:-y_min+h1, -x_min: -x_min+w1] = image1
    return image0_transformed_and_stitched

In [15]:
def TransformImage(image, H): 

    image0 = image.copy()
    h0 ,w0 ,_ = image0.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)

    for p in range(len(points_on_image0_transformed)):
        points_on_image0_transformed_.append(points_on_image0_transformed[p].ravel())

    points_on_image0_transformed_ = np.array(points_on_image0_transformed_)

    x_min, y_min = np.int0(np.min(points_on_image0_transformed, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_image0_transformed, axis = 0))
    
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    image0_transformed[-y_min:-y_min+h1, -x_min: -x_min+w1] = image1
    return image0_transformed

In [16]:
def cropImage(image):
    
    img = image.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(gray,5,255,cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[len(contours)-1])
    crop = img[y:y+h,x:x+w]

    return crop


In [25]:
def joinImages(img_array, choice, save_folder_name, n, show_steps = True):

    image_array = img_array.copy()
    N = len(image_array)
    image0 = image_array[0]
    for i in range(1, N):

        print("processing image ", i)
        image1 = image_array[i] 

        image_pair = [image0, image1]
        
        detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
        if show_steps:
            displayImages(corner_images, save_folder_name + "/corners" + str(n) + ".png")
        """
        Perform ANMS: Adaptive Non-Maximal Suppression
        Save ANMS output as anms.png
        """
        if (choice == 1):
            print("Applying ALMS.")
            detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
            if show_steps:
                displayImages(anms_image, save_folder_name + "/anms_output" + str(n) + ".png")
        else:
            print("goodFeaturesToTrack is already using ALMS.") #review

        detected_corners0 = detected_corners[0]
        detected_corners1 = detected_corners[1]
                
        matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
        if show_steps:
            showMatches(image0, image1, matched_pairs, save_folder_name + "/matched_pairs" + str(n) + ".png")
        """
        Refine: RANSAC, Estimate Homography
        """
        H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 5)
        if show_steps:
            showMatches(image0, image1, filtered_matched_pairs, save_folder_name + "/filtered_matched_pairs" + str(n) + ".png")
        """
        Image Warping + Blending
        Save Panorama output as mypano.png
        """
        stitched_image = stitchImagePairs(image0, image1, H)
        stitched_image = cropImage(stitched_image)
        if show_steps:
            cv2.imshow(save_folder_name + "/pano" + str(n) + ".png", stitched_image)
            cv2.waitKey() 
            cv2.destroyAllWindows()

        cv2.imwrite(save_folder_name + "/pano" + str(n) + ".png", stitched_image)
        image0 = stitched_image

    return image0


In [65]:
def autoPano():

    # Parser = argparse.ArgumentParser()
    # Parser.add_argument('--BasePath', default='/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/', help='base path')
    # Parser.add_argument('--ImagesFolder', default='Data/Train/Set1', help='folder for images')
    # Parser.add_argument('--SaveFolderName', default='Code/Results/Set1', help='Folder to save results')
    # Parser.add_argument('--ShowImages', type = bool, default= True, help='show images or not')
    # Args = Parser.parse_args()
    BasePath = '/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/'
    ImagesFolder = 'Data/Test/TestSet4'
    SaveFolderName = 'Code/Results/TestSet4'
    ShowImages = True

    use_harris = False

    images = readImageSet(BasePath + ImagesFolder)
    if ShowImages:
        displayImages(images, BasePath + ImagesFolder + "/input.png")

    N_images = len(images)

    choice = 2
    if use_harris:
        choice = 1

    N_first_half = round(N_images/2)
    N_second_half = N_images - N_first_half
    print(N_first_half, " and ", N_second_half)

    N_first_half_images = []
    N_second_half_images = []

    while N_images is not 2:
        print("N = ", N_images, " N_half = ", N_first_half)
        merged_images = []
        for n in range(0, N_first_half, 2):
            if (n+1) <= N_first_half:
                img_array = images[n:n+2]
                print("combining: ", n, n+1)
                I = joinImagesTest(img_array, choice, BasePath + SaveFolderName, n, ShowImages)
                merged_images.append(I)
            else:
                print("adding: ", n)
                merged_images.append(images[n])

        for n in range(N_first_half, N_images, 2):
            if (n+1) < N_images:
                img_array = images[n:n+2]
                img_array.reverse()
                print("combining: ", n+1, n)
                I = joinImagesTest(img_array, choice, BasePath + SaveFolderName, n, ShowImages)
                merged_images.append(I)
            else:
                print("adding: ", n, n)
                merged_images.append(images[n])
    
        images = merged_images
        N_images = len(images)
        N_first_half = round(N_images/2)
        N_second_half = N_images - N_first_half

    print("final merging")
    
    final = joinImages(merged_images, choice, BasePath + SaveFolderName, 100, ShowImages)

In [66]:
autoPano()

Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Test/TestSet4
Found  ['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']
2  and  3
N =  5  N_half =  2
combining:  0 1
processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  1280 , height =  720
width =  1280 , height =  720
all corner 1 len =  840
all corner 2 len =  1000
matched pairs num =  735
iterations =  4000
Number of pairs after filtering =  419
inbuilt =  [[ 1.11664529e+00 -1.68849734e-01 -1.27425600e+01]
 [ 1.53870342e+00  7.24606834e-01 -1.07743686e+03]
 [ 1.55526845e-03 -1.04189769e-03  1.00000000e+00]]
Computed =  [[ 1.18315372e+00  1.02255879e-02 -3.29876612e+02]
 [ 1.76009871e-01  1.11827380e+00 -5.26916744e+01]
 [ 2.71285227e-04 -9.58801623e-06  1.00000000e+00]]
H matrix is:  [[ 1.18315372e+00  1.02255879e-02 -3.29876612e+02]
 [ 1.76009871e-01  1.11827380e+00 -5.26916744e+01]
 [ 2.

In [27]:


# Parser = argparse.ArgumentParser()
# Parser.add_argument('--BasePath', default='/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/', help='base path')
# Parser.add_argument('--ImagesFolder', default='Data/Train/Set1', help='folder for images')
# Parser.add_argument('--SaveFolderName', default='Code/Results/Set1', help='Folder to save results')
# Parser.add_argument('--ShowImages', type = bool, default= True, help='show images or not')
# Args = Parser.parse_args()
BasePath = '/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/'
ImagesFolder = 'Data/Test/TestSet4'
SaveFolderName = 'Code/Results/TestSet4'
ShowImages = True

use_harris = False

images = readImageSet(BasePath + ImagesFolder)
if ShowImages:
    displayImages(images, BasePath + ImagesFolder + "/input.png")

N_images = len(images)

choice = 2
if use_harris:
    choice = 1

N_first_half = round(N_images/2)
N_second_half = N_images - N_first_half
print(N_first_half, " and ", N_second_half)

N_first_half_images = []
N_second_half_images = []



Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Test/TestSet4
Found  ['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']
2  and  3


In [28]:
img_array1 = [images[0], images[1]] 
I1 = joinImagesTest(img_array1, choice, BasePath + SaveFolderName, 1, ShowImages)


processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  1280 , height =  720
width =  1280 , height =  720
all corner 1 len =  840
all corner 2 len =  1000
matched pairs num =  735
iterations =  4000
Number of pairs after filtering =  417
inbuilt =  [[ 1.11664529e+00 -1.68849734e-01 -1.27425600e+01]
 [ 1.53870342e+00  7.24606834e-01 -1.07743686e+03]
 [ 1.55526845e-03 -1.04189769e-03  1.00000000e+00]]
Computed =  [[ 1.13541288e+00  8.04157861e-04 -3.08622403e+02]
 [ 1.60299578e-01  1.04864193e+00 -2.79047453e+01]
 [ 2.48030977e-04 -4.59507568e-05  1.00000000e+00]]
H matrix is:  [[ 1.13541288e+00  8.04157861e-04 -3.08622403e+02]
 [ 1.60299578e-01  1.04864193e+00 -2.79047453e+01]
 [ 2.48030977e-04 -4.59507568e-05  1.00000000e+00]]
shapes
(1280, 720, 3)
(1280, 720, 3)
transformed points =  [[[-308.6224    -27.904745]]

 [[-326.81537  1396.4945  ]]

 [[ 455.367    1276

In [59]:
img_array2 = [images[2], images[3]] 
I2 = joinImagesTest(img_array2, choice, BasePath + SaveFolderName, 1, ShowImages)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  1280 , height =  720
width =  1280 , height =  720
all corner 1 len =  950
all corner 2 len =  577
matched pairs num =  815
iterations =  4000
Number of pairs after filtering =  37
inbuilt =  [[-4.69883505e-01  5.20023413e-01 -3.23559129e+02]
 [ 2.08822245e+00 -2.25501959e+00  1.37556577e+03]
 [ 1.54597284e-03 -1.65302212e-03  1.00000000e+00]]
Computed =  [[ 1.58558737e-01 -6.57284667e-01  2.58201926e+02]
 [ 4.78583998e-01 -1.96426717e+00  7.69763036e+02]
 [ 6.15230961e-04 -2.54690211e-03  1.00000000e+00]]
Cannot match image
H matrix is:  [[ 1.58558737e-01 -6.57284667e-01  2.58201926e+02]
 [ 4.78583998e-01 -1.96426717e+00  7.69763036e+02]
 [ 6.15230961e-04 -2.54690211e-03  1.00000000e+00]]


In [62]:
cv2.imshow("h", I2)
cv2.waitKey() 
cv2.destroyAllWindows()

In [40]:
detected_corners, cmaps, corner_images = detectCorners(img_array2, choice)
if show_steps:
    displayImages(corner_images, save_folder_name + "/corners" + str(n) + ".png")

detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.


In [41]:
detected_corners0 = detected_corners[0]
detected_corners1 = detected_corners[1]
        
matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
if show_steps:
    showMatches(image0, image1, matched_pairs, save_folder_name + "/matched_pairs" + str(n) + ".png")


"""
Refine: RANSAC, Estimate Homography
"""
H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 15)


width =  1280 , height =  720
width =  1280 , height =  720
all corner 1 len =  950
all corner 2 len =  577
matched pairs num =  815
iterations =  4000
Number of pairs after filtering =  35
inbuilt =  [[-4.69883505e-01  5.20023413e-01 -3.23559129e+02]
 [ 2.08822245e+00 -2.25501959e+00  1.37556577e+03]
 [ 1.54597284e-03 -1.65302212e-03  1.00000000e+00]]
Computed =  [[-4.64727721e-01 -4.53878352e-02  2.57825681e+02]
 [-1.38936753e+00 -1.36599605e-01  7.71654135e+02]
 [-1.79402473e-03 -1.79292332e-04  1.00000000e+00]]


In [57]:
unique, counts = np.unique(filtered_matched_pairs[:,1,:], return_counts=True, axis = 0)
unique_count = unique.shape[0]
max_count = np.max(counts)
if(unique_count < 10 and max_count > 8):
    print("Cannot match image")

Cannot match image


In [20]:
img_array2 = [images[1], images[2]] 
I2 = joinImages(img_array2, choice, BasePath + SaveFolderName, 1, ShowImages)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  758 , height =  568
width =  758 , height =  568
all corner 1 len =  1000
all corner 2 len =  860
matched pairs num =  927
iterations =  4000
Number of pairs after filtering =  204
inbuilt =  [[ 7.79165636e-01 -7.99255581e-01  1.71679301e+02]
 [ 2.51460020e-01 -2.47461313e-01  4.84423216e+01]
 [ 4.38525202e-03 -4.56923847e-03  1.00000000e+00]]
Computed =  [[ 1.35816735e+00 -2.34412049e-01 -3.82199202e+02]
 [ 6.38612987e-01  1.19911801e+00 -2.15666547e+02]
 [ 9.47435977e-04 -2.15751764e-04  1.00000000e+00]]
shapes
(758, 568, 3)
(758, 568, 3)
transformed points =  [[[-382.1992  -215.66655]]

 [[-669.3487   828.80804]]

 [[ 153.90291  768.21924]]

 [[ 253.0582    95.61242]]]
min, max
-669 -215
568 828


In [26]:
u =np.where(I2 == [0,0,0])

In [31]:
u[0] + 10

array([  10,   10,   10, ..., 1052, 1052, 1052])

In [29]:
img_array3 = [I1, I2] 
I3 = joinImagesTest(img_array3, choice, BasePath + SaveFolderName, 1, ShowImages)

NameError: name 'I2' is not defined

In [21]:
def stitchImagePairsTest(img0, img1, H):

    image0 = img0.copy()
    image1 = img1.copy()

    #stitch image 0 on image 1
    print("shapes")
    print(image0.shape)
    print(image1.shape)
    

    h0 ,w0 ,_ = image0.shape
    h1 ,w1 ,_ = image1.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)
    print("transformed points = ", points_on_image0_transformed)
    points_on_image1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1,1,2)

    points_on_merged_images = np.concatenate((points_on_image0_transformed, points_on_image1), axis = 0)
    points_on_merged_images_ = []

    for p in range(len(points_on_merged_images)):
        points_on_merged_images_.append(points_on_merged_images[p].ravel())

    points_on_merged_images_ = np.array(points_on_merged_images_)

    x_min, y_min = np.int0(np.min(points_on_merged_images_, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_merged_images_, axis = 0))

   
    print("min x.y = ", x_min, y_min)
    print("max x,y = ", x_max, y_max)

    # overlap_area = cv2.polylines(image1,[np.int32(points_on_image0_transformed)],True,255,3, cv2.LINE_AA) 
    # cv2.imshow("original_image_overlapping.jpg", overlap_area)
    # cv2.waitKey() 
    # cv2.destroyAllWindows()
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed_and_stitched = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    print("h1 = ", h1, "w1 = ", w1)

    # images_stitched = image0_transformed_and_stitched.copy()
    image0_transformed_and_stitched[-y_min:-y_min+h1, -x_min:-x_min+w1] = image1

    # indices = np.where(image1 == [0,0,0])
    # y = indices[0] + -y_min 
    # x = indices[1] + -x_min 

    # images_stitched[y,x] = image0_transformed_and_stitched[y,x]
                
    return image0_transformed_and_stitched

In [58]:
def joinImagesTest(img_array, choice, save_folder_name, n, show_steps = True):

    image_array = img_array.copy()
    N = len(image_array)
    image0 = image_array[0]
    for i in range(1, N):

        print("processing image ", i)
        image1 = image_array[i] 

        image_pair = [image0, image1]
        
        detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
        if show_steps:
            displayImages(corner_images, save_folder_name + "/corners" + str(n) + ".png")
        """
        Perform ANMS: Adaptive Non-Maximal Suppression
        Save ANMS output as anms.png
        """
        if (choice == 1):
            print("Applying ALMS.")
            detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
            if show_steps:
                displayImages(anms_image, save_folder_name + "/anms_output" + str(n) + ".png")
        else:
            print("goodFeaturesToTrack is already using ALMS.") #review

        detected_corners0 = detected_corners[0]
        detected_corners1 = detected_corners[1]
                
        matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
        if show_steps:
            showMatches(image0, image1, matched_pairs, save_folder_name + "/matched_pairs" + str(n) + ".png")

        
        """
        Refine: RANSAC, Estimate Homography
        """
        H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 15)

        #remove the points that converge to a single point

        if show_steps:
            showMatches(image0, image1, filtered_matched_pairs, save_folder_name + "/filtered_matched_pairs" + str(n) + ".png")

        unique, counts = np.unique(filtered_matched_pairs[:,1,:], return_counts=True, axis = 0)
        unique_count = unique.shape[0]
        max_count = np.max(counts)
        stitching = True
        if(unique_count < 10 and max_count > 8):
            print("Cannot match image")
            stitching = False


        """
        Image Warping + Blending
        Save Panorama output as mypano.png
        """
        print("H matrix is: ", H)
        if(stitching):
            stitched_image = stitchImagePairsTest(image0, image1, H)
            print("SIZEEEEEEE = ",stitched_image.shape )
            stitched_image = cropImage(stitched_image)
            if show_steps:
                cv2.imshow(save_folder_name + "/pano" + str(n) + ".png", stitched_image)
                cv2.waitKey() 
                cv2.destroyAllWindows()

            cv2.imwrite(save_folder_name + "/pano" + str(n) + ".png", stitched_image)
            image0 = stitched_image

    return image0


In [24]:
if __name__ == '__main__':
    autoPano()

Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set3
Found  ['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg']
4  and  4
N =  8  N_half =  4
combining:  0 1
processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  489
all corner 2 len =  561
matched pairs num =  442
iterations =  4000
Number of pairs after filtering =  109
inbuilt =  [[ 8.98932359e-01 -8.09269316e-01  2.35080040e+02]
 [ 1.94672202e+00 -1.72142241e+00  4.96172856e+02]
 [ 3.89482378e-03 -3.46407303e-03  1.00000000e+00]]
Computed =  [[ 1.42847686e+00 -7.05199963e-02 -3.86733963e+02]
 [ 2.96495746e-01  1.48522195e+00 -1.62315629e+02]
 [ 8.58048285e-04  6.42618054e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[-386.73398  -162.31563 ]]

 [[-421.77087